## Create Universe

In [1]:
import MDAnalysis as mda

TOPOL = "/scratch/ljwoods2/workspace/zarrtraj/zarrtraj/data/yiip_equilibrium/YiiP_system.pdb"
TRAJ = "/scratch/ljwoods2/workspace/zarrtraj/zarrtraj/data/yiip_aligned_compressed.h5md"

u = mda.Universe(TOPOL, TRAJ)

/nfs/homes3/ljwoods2/miniforge3/envs/mda-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/nfs/homes3/ljwoods2/workspace/mdanalysis/package/MDAnalysis/topology/PDBParser.py:348: UserWarning: Unknown element Z found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/nfs/homes3/ljwoods2/workspace/mdanalysis/package/MDAnalysis/topology/PDBParser.py:348: UserWarning: Unknown element D found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/nfs/homes3/ljwoods2/workspace/mdanalysis/package/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.w

## Run dask RMSF method

In [2]:
from MDAnalysis.analysis.dasktimeseries import DaskRMSF

dask_timeseries = u.trajectory.dask_timeseries()
dRMSF = DaskRMSF(dask_timeseries).run()
dRMSF_result = dRMSF.results.rmsf

## Run RMSF method shipped with mda

In [5]:
from MDAnalysis.analysis.rms import RMSF

RMSF = RMSF(u.atoms).run()
RMSF_result = RMSF.results.rmsf

## Ensure results are the same

In [8]:
from numpy.testing import assert_allclose

assert_allclose(dRMSF_result, RMSF_result, atol=1e-5)

## Print num cores used by dask scheduler

In [15]:
import dask
import multiprocessing
num_threads = dask.config.get('num_workers', 0)
print(num_threads)

KeyError: 'num_workers'

## Settings used to write the trajectory used above

In [ ]:
import MDAnalysis as mda
import MDAnalysisData
from MDAnalysis.analysis import align

yiip = MDAnalysisData.yiip_equilibrium.fetch_yiip_equilibrium_long()

u = mda.Universe(
    yiip.topology,
    yiip.trajectory
)

average = align.AverageStructure(
    u, u, select="protein and name CA", ref_frame=0
).run()
ref = average.results.universe

# Writer kwargs only passable to align in mda 2.8.0
# to relax requirement, just write in two steps

# 1. Align traj and write to xtc
aligner = align.AlignTraj(
    u,
    ref,
    select="protein and name CA",
    filename="/scratch/ljwoods2/workspace/zarrtraj/data/yiip_equilibrium/YiiP_system_90ns_center_aligned.xtc",
).run()

# 2. Write aligned xtc traj to H5MD
u_aligned = mda.Universe(
    yiip.topology,
    "/scratch/ljwoods2/workspace/zarrtraj/data/yiip_equilibrium/YiiP_system_90ns_center_aligned.xtc"
)

with mda.Writer(
    "/scratch/ljwoods2/workspace/zarrtraj/zarrtraj/data/yiip_aligned_compressed.h5md",
    n_atoms=u_aligned.trajectory.n_atoms,
    n_frames=u_aligned.trajectory.n_frames,
    compression="gzip",
    compression_opts=9,
    chunks=(9, u.trajectory.n_atoms, 3),
) as W:
    for ts in u_aligned.trajectory:
        W.write(u_aligned.atoms)
